In [2]:
import pandas as pd 
import os

os.chdir("/Volumes/SAMSUNG/trpred")

# Get all trp users from submissions
users_submissions = pd.read_csv("data/processed/submissions/TheRedPill-allsubmissions-2020-05-16-metadata.csv", usecols = ["id","author"])

# Get submission ids to search for author flair by submission
ls_subids = list(users_submissions.id)


['9c7knc',
 '9c7ib2',
 '9c79bf',
 '9c5afj',
 '9c57us',
 '9c3jg2',
 '9c3d9c',
 '9c3ane',
 '9c37kt',
 '9c35e5',
 '9c31xi',
 '9c2ahn',
 '9c26x2',
 '9c1pq2',
 '9c1n1d',
 '9c1ev3',
 '9c1dpy',
 '9c1bdj',
 '9c0ela',
 '9c08lt',
 '9bz91j',
 '9byyho',
 '9bypsk',
 '9byn6i',
 '9bymsi',
 '9bymki',
 '9bye5f',
 '9byblx',
 '9by34z',
 '9bxu5u',
 '9bx0e6',
 '9bwqp6',
 '9bw9e6',
 '9bw2nb',
 '9bvoq7',
 '9bv0mi',
 '9bu265',
 '9btvux',
 '9btfhx',
 '9bt36d',
 '9bt2go',
 '9bssz9',
 '9bs95p',
 '9bs84r',
 '9bs1d5',
 '9brxd9',
 '9brqoo',
 '9br6pi',
 '9br4ez',
 '9bqgue',
 '9bqed9',
 '9boiff',
 '9boerw',
 '9boaba',
 '9bnyzh',
 '9bntof',
 '9bnhf6',
 '9bnf3g',
 '9bnena',
 '9bn6p1',
 '9bmmlf',
 '9bmm7e',
 '9bmm4g',
 '9bmcvm',
 '9bma1u',
 '9bm56y',
 '9bm1a1',
 '9blp76',
 '9blp2l',
 '9bloov',
 '9blj3a',
 '9bl5v4',
 '9bl18v',
 '9bkr6k',
 '9bjzqz',
 '9bjviq',
 '9bjss1',
 '9bjsix',
 '9bjre4',
 '9bji2v',
 '9bje1y',
 '9bj3vn',
 '9bj24c',
 '9biqjh',
 '9bi7po',
 '9bgs4f',
 '9bghk6',
 '9bg3wu',
 '9bfz65',
 '9bfb83',
 '9bezqa',

In [4]:
test_id = "993wvk"
test_id

'993wvk'

In [5]:
# https://api.pushshift.io/reddit/submission/search/?ids=9c7knc
import requests 

url = "https://api.pushshift.io/reddit/submission/search/?ids="
url = url + test_id

url


'https://api.pushshift.io/reddit/submission/search/?ids=993wvk'

In [6]:
# initializing this here so it doesn't restart every time i rerun it
trpuser_flairs = {}

In [ ]:
import numpy as np 
import requests 
import time 
import pandas as pd
import json
import os 

os.chdir("/Volumes/SAMSUNG/trpred")

counter = 0
filename = "data/info/20201004_trp_use_flairs.json"

for i in ls_subids:
    # set url to search by submission id
    url = "https://api.pushshift.io/reddit/submission/search/?ids="
    url = url + i # concat id from list
    r = requests.get(url) # make request
    
    author = users_submissions.loc[users_submissions['id'] == i]['author']
    author = author.values[0]
    
    try:
        flair = r.json()['data'][0]['author_flair_text'] # get flair if it exists
    except:
        flair = np.nan # otherwise, no flair
    
    trpuser_flairs[author] = flair
    
    print(i)
    counter += 1 
    print("og start +:", counter)
    
    time.sleep(1)

    if len(trpuser_flairs.keys()) % 1000 == 0:
        try:
            with open(filename, 'wb') as outfile:
                json.dump(trpuser_flairs, outfile)
        except:
            continue

    
    
    
    
    

9c7knc
og start +: 1
9c7ib2
og start +: 2
9c79bf
og start +: 3
9c5afj
og start +: 4
9c57us
og start +: 5
9c3jg2
og start +: 6
9c3d9c
og start +: 7
9c3ane
og start +: 8
9c37kt
og start +: 9
9c35e5
og start +: 10
9c31xi
og start +: 11
9c2ahn
og start +: 12
9c26x2
og start +: 13
9c1pq2
og start +: 14
9c1n1d
og start +: 15
9c1ev3
og start +: 16
9c1dpy
og start +: 17
9c1bdj
og start +: 18
9c0ela
og start +: 19
9c08lt
og start +: 20
9bz91j
og start +: 21
9byyho
og start +: 22
9bypsk
og start +: 23
9byn6i
og start +: 24
9bymsi
og start +: 25
9bymki
og start +: 26
9bye5f
og start +: 27
9byblx
og start +: 28
9by34z
og start +: 29
9bxu5u
og start +: 30
9bx0e6
og start +: 31
9bwqp6
og start +: 32
9bw9e6
og start +: 33
9bw2nb
og start +: 34
9bvoq7
og start +: 35
9bv0mi
og start +: 36
9bu265
og start +: 37
9btvux
og start +: 38
9btfhx
og start +: 39
9bt36d
og start +: 40
9bt2go
og start +: 41
9bssz9
og start +: 42
9bs95p
og start +: 43
9bs84r
og start +: 44
9bs1d5
og start +: 45
9brxd9
og start +: 

99bwb2
og start +: 362
99bsbd
og start +: 363
99bbnd
og start +: 364
99b9ee
og start +: 365
99b9dj
og start +: 366
99b8by
og start +: 367
99anl3
og start +: 368
99al4e
og start +: 369
99akt8
og start +: 370
99aijc
og start +: 371
99aht9
og start +: 372
99ahaq
og start +: 373
99agff
og start +: 374
99afao
og start +: 375
99ab7r
og start +: 376
99a21d
og start +: 377
999y5i
og start +: 378
999k5m
og start +: 379
999ihb
og start +: 380
999bus
og start +: 381
999ada
og start +: 382
9995pw
og start +: 383
9990lg
og start +: 384
998xsk
og start +: 385
998urx
og start +: 386
998bfx
og start +: 387
997eyh
og start +: 388
997a3p
og start +: 389
997a2r
og start +: 390
996xdr
og start +: 391
996k8h
og start +: 392
996f5c
og start +: 393
996a5q
og start +: 394
9969cq
og start +: 395
9963h1
og start +: 396
995ya2
og start +: 397
995w4u
og start +: 398
995vuo
og start +: 399
995hct
og start +: 400
995gwi
og start +: 401
995chf
og start +: 402
995bko
og start +: 403
994pr2
og start +: 404
994bcy
og s

9754h3
og start +: 719
974wdr
og start +: 720
974so8
og start +: 721
974rnk
og start +: 722
97480x
og start +: 723
973v7k
og start +: 724
973ufx
og start +: 725
973qw8
og start +: 726
973l7b
og start +: 727
973jnw
og start +: 728
973ccl
og start +: 729
973by1
og start +: 730
972zql
og start +: 731
972ui3
og start +: 732
972sow
og start +: 733
972js0
og start +: 734
972iy9
og start +: 735
9725w3
og start +: 736
971iyf
og start +: 737
9719pd
og start +: 738
97186r
og start +: 739
971790
og start +: 740
970zb6
og start +: 741
970red
og start +: 742
970ovb
og start +: 743
970lt2
og start +: 744
970k8h
og start +: 745
970cjk
og start +: 746
9703hp
og start +: 747
96zvnc
og start +: 748
96zsnl
og start +: 749
96zgzw
og start +: 750
96zeyh
og start +: 751
96zcm2
og start +: 752
96yzai
og start +: 753
96yyz5
og start +: 754
96yvy0
og start +: 755
96yqec
og start +: 756
96y7f2
og start +: 757
96y5xe
og start +: 758
96y22r
og start +: 759
96xvrp
og start +: 760
96xrig
og start +: 761
96xixc
og s

94o6qx
og start +: 1073
94nux5
og start +: 1074
94nskq
og start +: 1075
94nnkn
og start +: 1076
94n951
og start +: 1077
94n5v2
og start +: 1078
94mrld
og start +: 1079
94mmvw
og start +: 1080
94mez8
og start +: 1081
94mcli
og start +: 1082
94mceu
og start +: 1083
94m43j
og start +: 1084
94lzdi
og start +: 1085
94lxfv
og start +: 1086
94luli
og start +: 1087
94liyu
og start +: 1088
94kvk3
og start +: 1089
94kt0w
og start +: 1090
94kq2r
og start +: 1091
94kkxs
og start +: 1092
94kea6
og start +: 1093
94kdse
og start +: 1094
94jyd8
og start +: 1095
94junc
og start +: 1096
94jac7
og start +: 1097
94iw8s
og start +: 1098
94ivq6
og start +: 1099
94iv3u
og start +: 1100
94irno
og start +: 1101
94imxs
og start +: 1102
94ieom
og start +: 1103
94hrgb
og start +: 1104
94gts4
og start +: 1105
94gn0z
og start +: 1106
94g672
og start +: 1107
94fzsf
og start +: 1108
94fyth
og start +: 1109
94fknk
og start +: 1110
94fkhe
og start +: 1111
94ffd0
og start +: 1112
94f21d
og start +: 1113
94exe0
og start 

92navi
og start +: 1415
92mgu4
og start +: 1416
92mcey
og start +: 1417
92m4ry
og start +: 1418
92m4bl
og start +: 1419
92lype
og start +: 1420
92lvef
og start +: 1421
92lrqa
og start +: 1422
92lgg0
og start +: 1423
92lg4a
og start +: 1424
92lf2k
og start +: 1425
92l3f4
og start +: 1426
92l0w6
og start +: 1427
92l0il
og start +: 1428
92kwhm
og start +: 1429
92kl47
og start +: 1430
92k96w
og start +: 1431
92k95x
og start +: 1432
92k6k3
og start +: 1433
92k0xy
og start +: 1434
92jt33
og start +: 1435
92j3bt
og start +: 1436
92ior5
og start +: 1437
92io1u
og start +: 1438
92ilhc
og start +: 1439
92ij45
og start +: 1440
92hvw4
og start +: 1441
92hrze
og start +: 1442
92hrhk
og start +: 1443
92hq30
og start +: 1444
92ho4e
og start +: 1445
92hiss
og start +: 1446
92hh80
og start +: 1447
92hgvf
og start +: 1448
92he8d
og start +: 1449
92gxpa
og start +: 1450
92goul
og start +: 1451
92gk9e
og start +: 1452
92gf0m
og start +: 1453
92gcyv
og start +: 1454
92gb87
og start +: 1455
92g901
og start 

...

61k8c7
og start +: 23087
61k2u4
og start +: 23088
61k2cc
og start +: 23089
61jzv8
og start +: 23090
61jr1r
og start +: 23091
61jp0k
og start +: 23092
61jh0q
og start +: 23093
61j5jw
og start +: 23094
61j36g
og start +: 23095
61j2mv
og start +: 23096
61j0yf
og start +: 23097
61ix22
og start +: 23098
61iwst
og start +: 23099
61iv64
og start +: 23100
61it4i
og start +: 23101
61it2l
og start +: 23102
61isv2
og start +: 23103
61is30
og start +: 23104
61ioux
og start +: 23105
61io5k
og start +: 23106
61imf8
og start +: 23107
61ilp9
og start +: 23108
61ijwi
og start +: 23109
61iij5
og start +: 23110
61i8nm
og start +: 23111
61i7g9
og start +: 23112
61i5k6
og start +: 23113
61i3mj
og start +: 23114
61i2by
og start +: 23115
61hvth
og start +: 23116
61hufs
og start +: 23117
61hllu
og start +: 23118
61hehq
og start +: 23119
61hclc
og start +: 23120
61haiw
og start +: 23121
61h2st
og start +: 23122
61h1q2
og start +: 23123
61h1nv
og start +: 23124
61gzxb
og start +: 23125
61gzix
og start +: 23126


60f90u
og start +: 23415
60f7l8
og start +: 23416
60f5nu
og start +: 23417
60ex78
og start +: 23418
60eqw3
og start +: 23419
60elqh
og start +: 23420
60ei1m
og start +: 23421
60eg29
og start +: 23422
60efvv
og start +: 23423
60edxv
og start +: 23424
60edqx
og start +: 23425
60e4uq
og start +: 23426
60dy8q
og start +: 23427
60dwed
og start +: 23428
60drkm
og start +: 23429
60drcv
og start +: 23430
60dm3v
og start +: 23431
60dl1b
og start +: 23432
60dj78
og start +: 23433
60dch4
og start +: 23434
60dbaj
og start +: 23435
60d8xw
og start +: 23436
60cvu1
og start +: 23437
60cnhb
og start +: 23438
60clro
og start +: 23439
60cjva
og start +: 23440
60cim8
og start +: 23441
60cf4o
og start +: 23442
60cc7a
og start +: 23443
60calu
og start +: 23444
60c4ut
og start +: 23445
60c11s
og start +: 23446
60bzbv
og start +: 23447
60by2m
og start +: 23448
60bs1j
og start +: 23449
60blqp
og start +: 23450
60bkr5
og start +: 23451
60bkk3
og start +: 23452
60bipd
og start +: 23453
60bgzx
og start +: 23454


5z9qy8
og start +: 23743
5z9jqe
og start +: 23744
5z9j7t
og start +: 23745
5z9i5p
og start +: 23746
5z9g7g
og start +: 23747
5z9fyl
og start +: 23748
5z9c9c
og start +: 23749
5z9b4i
og start +: 23750
5z92ju
og start +: 23751
5z8v2o
og start +: 23752
5z8hjf
og start +: 23753
5z8h5x
og start +: 23754
5z8anp
og start +: 23755
5z85zr
og start +: 23756
5z85sv
og start +: 23757
5z85kv
og start +: 23758
5z835y
og start +: 23759
5z7z7h
og start +: 23760
5z7uuy
og start +: 23761
5z7iyt
og start +: 23762
5z7f2q
og start +: 23763
5z7b60
og start +: 23764
5z79zc
og start +: 23765
5z78v4
og start +: 23766
5z74ez
og start +: 23767
5z6zqx
og start +: 23768
5z6k4e
og start +: 23769
5z6iu4
og start +: 23770
5z6ems
og start +: 23771
5z5wz5
og start +: 23772
5z5nyw
og start +: 23773
5z5mfp
og start +: 23774
5z5jmx
og start +: 23775
5z5d2r
og start +: 23776
5z5cfl
og start +: 23777
5z5a97
og start +: 23778
5z4vrn
og start +: 23779
5z4vh9
og start +: 23780
5z4rpm
og start +: 23781
5z4e5s
og start +: 23782


5y26dh
og start +: 24071
5y23rk
og start +: 24072
5y1r4d
og start +: 24073
5y1nln
og start +: 24074
5y142v
og start +: 24075
5y130i
og start +: 24076
5y11s0
og start +: 24077
5y0wb1
og start +: 24078
5y0kwa
og start +: 24079
5y0ivq
og start +: 24080
5y0hkp
og start +: 24081
5y0aav
og start +: 24082
5xzy2i
og start +: 24083
5xztvx
og start +: 24084
5xzqbz
og start +: 24085
5xzndj
og start +: 24086
5xzm6b
og start +: 24087
5xzm07
og start +: 24088
5xzjsk
og start +: 24089
5xze68
og start +: 24090
5xyyub
og start +: 24091
5xywoa
og start +: 24092
5xyvsb
og start +: 24093
5xysjd
og start +: 24094
5xyrkf
og start +: 24095
5xyo8h
og start +: 24096
5xyo65
og start +: 24097
5xynfv
og start +: 24098
5xydx4
og start +: 24099
5xycxu
og start +: 24100
5xyb20
og start +: 24101
5xy5vm
og start +: 24102
5xy599
og start +: 24103
5xy4qe
og start +: 24104
5xy08m
og start +: 24105
5xxvyj
og start +: 24106
5xxtzu
og start +: 24107
5xxnu0
og start +: 24108
5xxmz1
og start +: 24109
5xxlsg
og start +: 24110


5wxx24
og start +: 24399
5wxvdu
og start +: 24400
5wxu8r
og start +: 24401
5wxnz8
og start +: 24402
5wxnx5
og start +: 24403
5wxbdv
og start +: 24404
5wxagm
og start +: 24405
5wx46a
og start +: 24406
5wx143
og start +: 24407
5wwxrt
og start +: 24408
5www8o
og start +: 24409
5wwupd
og start +: 24410
5wwudf
og start +: 24411
5wwu9r
og start +: 24412
5wwrr2
og start +: 24413
5wwk9p
og start +: 24414
5wwj1r
og start +: 24415
5wwd53
og start +: 24416
5ww68r
og start +: 24417
5wvyng
og start +: 24418
5wvxbd
og start +: 24419
5wvn6y
og start +: 24420
5wv14c
og start +: 24421
5wuzmp
og start +: 24422
5wuzgp
og start +: 24423
5wuz4t
og start +: 24424
5wuua0
og start +: 24425
5wuu9n
og start +: 24426
5wu4xs
og start +: 24427
5wu3uq
og start +: 24428
5wtw9l
og start +: 24429
5wtu5m
og start +: 24430
5wtqgj
og start +: 24431
5wsx2y
og start +: 24432
5wsvcc
og start +: 24433
5wssfa
og start +: 24434
5wsklh
og start +: 24435
5wsi91
og start +: 24436
5wsi7m
og start +: 24437
5wsgoe
og start +: 24438


In [22]:
len(trpuser_flairs.keys())
len(ls_subids)

103703

In [ ]:
# Fix up the DF
df_flairs = pd.DataFrame.from_dict(subred_descriptions, orient = "index", columns = ["flair"])
df_flairs = df_flairs.reset_index()
df_flairs = df_flairs.rename(columns = {"index":"user"})


In [ ]:
# Save to csv
os.chdir("/Volumes/SAMSUNG/trpred")

df_flairs.to_csv("info/trp_use_flairs.csv", mode = "w")

In [37]:
import pandas as pd 
import os

os.chdir("/Volumes/SAMSUNG/trpred")

# Get all trp users from comments
users_comments = pd.read_csv("data/processed/comments/TheRedPill-comments-2020-06-01.csv", usecols = ["author", "author_flair_text"])

## NEED TO REPROCESS R/TRP COMMENTS BC OF STUPID ICLOUD



FileNotFoundError: [Errno 2] File data/processed/comments/TheRedPill-comments-2020-06-01.csv does not exist: 'data/processed/comments/TheRedPill-comments-2020-06-01.csv'

In [ ]:
users_comments

In [3]:
import pandas as pd
import re
import utils
import datetime as dt
import csv
import os
import b_clean_data as clean
from os import listdir
from os.path import isfile, join
from fsplit.filesplit import FileSplit
import glob
from os import listdir
from os.path import isfile, join

data/raw/comments/Braincels-allcomments-2020-10-02.json
Folder already created.
data/raw/comments/Braincels-allcomments-2020-10-02.json cleaned


In [5]:
comment_files[4]

'data/raw/comments/Braincels-allcomments-2020-10-02.json'

In [65]:

import pandas as pd
import re
import utils
import datetime as dt
import csv
import os
from os import listdir
from os.path import isfile, join
from fsplit.filesplit import FileSplit
import glob
from os import listdir
from os.path import isfile, join

def clean_comments(raw_comments):
    """Cleans comments found in the folder passed in."""

    new_folder = utils.create_folder(raw_comments)

    if new_folder == None:
        print("No directory created")
        new_folder = "data/raw/comments/" + utils.get_filename(raw_comments)

    # Get name for processed file
    print("raw_comments = " + raw_comments)

    
    regex = r"([^\/]+)(?=.allcomments)"
    matches = re.search(regex, raw_comments)
    new_file = matches.group(1)

    # Comment files
    files = [f for f in listdir(new_folder) if isfile(join(new_folder, f))]
    print(*files, sep = "\n")
    
    # Create list of columns to keep
    keep_cols = ['id', 'created_utc','author',\
                  'author_flair_text', 'score', 'parent_id',\
                  'subreddit']
    keep_cols_text = ['id', 'created_utc', 'parent_id', 'body']

    # Create file name
    processedfile_csv = "data/processed/comments/" + new_file + \
        "-metadata" +  ".csv"

    processed_textfile_csv = "data/processed/comments/" + new_file + \
        "-text" + ".csv"

    counter = 0

    # Read in json file
    for i in files:
        print(i)
        counter += 1

        df_keep = pd.DataFrame()
        df_keep_text = pd.DataFrame()

        file_path = new_folder + "/" + i

        try:
            data = pd.read_json(file_path)

        # ValueError: Trailing data thrown if file is pretty indented
        except ValueError:
            data = pd.read_json(file_path, lines = True)


        try:
            df_keep = df_keep.append(data[keep_cols])
        except KeyError:
            keep_cols = ['id', 'created_utc', 'author', 'title',\
                        'score', 'num_comments', 'subreddit']
            df_keep = df_keep.append(data[keep_cols])


        try:
            df_keep_text = df_keep_text.append(data[keep_cols_text])
        except KeyError:
            keep_cols_text = ['id', 'created_utc', 'author']
            df_keep_text = df_keep_text.append(data[keep_cols_text])

        # Make sure there's at least 1 observation
        observations = len(df_keep.index)

        # Change date format
        ## For metadata
        if observations == 0:
            print("No comments found in " + i)
            continue

        else:
            df_keep['datetime_dv'] = pd.to_datetime(df_keep['created_utc'], unit = 's')# dv = derived
            df_keep['date_dv'] = df_keep['datetime_dv'].dt.date

            # For text
            df_keep_text['datetime_dv'] = pd.to_datetime(df_keep_text['created_utc'], unit = 's')# dv = derived
            df_keep_text['date_dv'] = df_keep_text['datetime_dv'].dt.date


        ##### Delimit by date #####
        start = dt.datetime.fromtimestamp(utils.get_startdate()).date()
        end = dt.datetime.fromtimestamp(utils.get_enddate()).date()

        # Create mask of time slot
        mask = (df_keep['date_dv'] >= start) & (df_keep['date_dv'] <= end) # inclusive on either end
        mask_text = (df_keep_text['date_dv'] >= start) & (df_keep_text['date_dv'] <= end)

        # Only keep data within date frame
        df_keep = df_keep.loc[mask]
        df_keep_text = df_keep_text.loc[mask_text]
        ############################

        # Save to csv
        if counter == 1:
            df_keep.to_csv(processedfile_csv, mode = "w")
            df_keep_text.to_csv(processed_textfile_csv, mode = "w")

        else:
            df_keep.to_csv(processedfile_csv, mode = "a", header = False)
            df_keep_text.to_csv(processed_textfile_csv, mode = "a", header = False)







In [70]:
## Clean comments and submissions
os.chdir("/Volumes/SAMSUNG/trpred")# Get comment folders

# comment_folders = [x[0] for x in os.walk("data/raw/comments")]
# comment_folders = comment_folders[1:] # item at index 0 is parent folder so exclude

comment_files = glob.glob("data/raw/comments/*.json")


# comment_files = ["data/raw/comments/Braincels-allcomments-2020-10-02.json",
# "data/raw/comments/exredpill-allcomments-2020-10-03.json",
# "data/raw/comments/fPUA-allcomments-2020-10-03.json",
# "data/raw/comments/IncelTears-allcomments-2020-10-03.json",
# "data/raw/comments/KotakuInAction-allcomments-2020-10-06.json",
# "data/raw/comments/masculism-allcomments-2020-10-03.json",
# "data/raw/comments/MensRights-allcomments-2020-10-02.json",
# "data/raw/comments/MGTOW-allcomments-2020-10-03.json",
# "data/raw/comments/PurplePillDebate-allcomments-2020-10-03.json",
# "data/raw/comments/TheRedPill-allcomments-2020-10-01.json"]


for i in comment_files:
    print(i)
    clean_comments(i)
    print(i + " end")
    
    
### NEW FILE ISNT BEING CREATED -- CHECK WHY

data/raw/comments/100sets-allcomments-2020-09-23.json
Creation of the directory data/raw/comments/100sets-allcomments-2020-09-23 failed
No directory created
raw_comments = data/raw/comments/100sets-allcomments-2020-09-23.json
100sets-allcomments-2020-09-23_1.json
100sets-allcomments-2020-09-23_1.json
data/raw/comments/100sets-allcomments-2020-09-23.json end
data/raw/comments/AskFeminists-allcomments-2020-08-20.json
Creation of the directory data/raw/comments/AskFeminists-allcomments-2020-08-20 failed
No directory created
raw_comments = data/raw/comments/AskFeminists-allcomments-2020-08-20.json
AskFeminists-allcomments-2020-08-20_1.json
AskFeminists-allcomments-2020-08-20_10.json
AskFeminists-allcomments-2020-08-20_11.json
AskFeminists-allcomments-2020-08-20_12.json
AskFeminists-allcomments-2020-08-20_13.json
AskFeminists-allcomments-2020-08-20_14.json
AskFeminists-allcomments-2020-08-20_15.json
AskFeminists-allcomments-2020-08-20_16.json
AskFeminists-allcomments-2020-08-20_17.json
AskF

Braincels-allcomments-2020-10-02_10.json
Braincels-allcomments-2020-10-02_100.json
Braincels-allcomments-2020-10-02_101.json
Braincels-allcomments-2020-10-02_102.json
Braincels-allcomments-2020-10-02_103.json
Braincels-allcomments-2020-10-02_104.json
Braincels-allcomments-2020-10-02_105.json
Braincels-allcomments-2020-10-02_106.json
Braincels-allcomments-2020-10-02_107.json
Braincels-allcomments-2020-10-02_108.json
Braincels-allcomments-2020-10-02_109.json
Braincels-allcomments-2020-10-02_11.json
Braincels-allcomments-2020-10-02_110.json
Braincels-allcomments-2020-10-02_111.json
Braincels-allcomments-2020-10-02_112.json
Braincels-allcomments-2020-10-02_113.json
Braincels-allcomments-2020-10-02_114.json
Braincels-allcomments-2020-10-02_115.json
Braincels-allcomments-2020-10-02_116.json
Braincels-allcomments-2020-10-02_117.json
Braincels-allcomments-2020-10-02_118.json
Braincels-allcomments-2020-10-02_119.json
Braincels-allcomments-2020-10-02_12.json
Braincels-allcomments-2020-10-02_120.

FeMRADebates-allcomments-2020-08-20_10.json
FeMRADebates-allcomments-2020-08-20_11.json
FeMRADebates-allcomments-2020-08-20_12.json
FeMRADebates-allcomments-2020-08-20_13.json
FeMRADebates-allcomments-2020-08-20_14.json
FeMRADebates-allcomments-2020-08-20_15.json
FeMRADebates-allcomments-2020-08-20_16.json
FeMRADebates-allcomments-2020-08-20_17.json
FeMRADebates-allcomments-2020-08-20_18.json
FeMRADebates-allcomments-2020-08-20_19.json
FeMRADebates-allcomments-2020-08-20_2.json
FeMRADebates-allcomments-2020-08-20_20.json
FeMRADebates-allcomments-2020-08-20_21.json
FeMRADebates-allcomments-2020-08-20_22.json
FeMRADebates-allcomments-2020-08-20_23.json
FeMRADebates-allcomments-2020-08-20_24.json
FeMRADebates-allcomments-2020-08-20_25.json
FeMRADebates-allcomments-2020-08-20_26.json
FeMRADebates-allcomments-2020-08-20_27.json
FeMRADebates-allcomments-2020-08-20_28.json
FeMRADebates-allcomments-2020-08-20_29.json
FeMRADebates-allcomments-2020-08-20_3.json
FeMRADebates-allcomments-2020-08-2

IncelsWithoutHate-allcomments-2020-08-20_10.json
IncelsWithoutHate-allcomments-2020-08-20_11.json
IncelsWithoutHate-allcomments-2020-08-20_12.json
IncelsWithoutHate-allcomments-2020-08-20_13.json
IncelsWithoutHate-allcomments-2020-08-20_14.json
IncelsWithoutHate-allcomments-2020-08-20_15.json
IncelsWithoutHate-allcomments-2020-08-20_16.json
IncelsWithoutHate-allcomments-2020-08-20_17.json
IncelsWithoutHate-allcomments-2020-08-20_18.json
IncelsWithoutHate-allcomments-2020-08-20_19.json
IncelsWithoutHate-allcomments-2020-08-20_2.json
IncelsWithoutHate-allcomments-2020-08-20_20.json
IncelsWithoutHate-allcomments-2020-08-20_21.json
IncelsWithoutHate-allcomments-2020-08-20_22.json
IncelsWithoutHate-allcomments-2020-08-20_23.json
IncelsWithoutHate-allcomments-2020-08-20_24.json
IncelsWithoutHate-allcomments-2020-08-20_25.json
IncelsWithoutHate-allcomments-2020-08-20_26.json
IncelsWithoutHate-allcomments-2020-08-20_27.json
IncelsWithoutHate-allcomments-2020-08-20_28.json
IncelsWithoutHate-all

KotakuInAction_allcomments_10-06_10.json
KotakuInAction_allcomments_10-06_100.json
KotakuInAction_allcomments_10-06_101.json
KotakuInAction_allcomments_10-06_102.json
KotakuInAction_allcomments_10-06_103.json
KotakuInAction_allcomments_10-06_104.json
KotakuInAction_allcomments_10-06_105.json
KotakuInAction_allcomments_10-06_106.json
KotakuInAction_allcomments_10-06_107.json
KotakuInAction_allcomments_10-06_108.json
KotakuInAction_allcomments_10-06_109.json
KotakuInAction_allcomments_10-06_11.json
KotakuInAction_allcomments_10-06_110.json
KotakuInAction_allcomments_10-06_111.json
KotakuInAction_allcomments_10-06_112.json
KotakuInAction_allcomments_10-06_113.json
KotakuInAction_allcomments_10-06_114.json
KotakuInAction_allcomments_10-06_115.json
KotakuInAction_allcomments_10-06_116.json
KotakuInAction_allcomments_10-06_117.json
KotakuInAction_allcomments_10-06_118.json
KotakuInAction_allcomments_10-06_119.json
KotakuInAction_allcomments_10-06_12.json
KotakuInAction_allcomments_10-06_120.

...

TumblrInAction-allcomments-2020-09-27_99.json
data/raw/comments/TumblrInAction-allcomments-2020-09-27.json end
data/raw/comments/WhereAreAllTheGoodMen-allcomments-2020-09-15.json
Creation of the directory data/raw/comments/WhereAreAllTheGoodMen-allcomments-2020-09-15 failed
No directory created
raw_comments = data/raw/comments/WhereAreAllTheGoodMen-allcomments-2020-09-15.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_1.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_10.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_11.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_12.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_13.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_14.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_15.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_16.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_17.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_18.json
WhereAreAllTheGoodMen-allcomments-2020-09-15_19.json
WhereAreAllTheGoodMen-allcomments-2020-

askTRP-allcomments-2020-08-19_10.json
askTRP-allcomments-2020-08-19_100.json
askTRP-allcomments-2020-08-19_101.json
askTRP-allcomments-2020-08-19_102.json
askTRP-allcomments-2020-08-19_103.json
askTRP-allcomments-2020-08-19_104.json
askTRP-allcomments-2020-08-19_105.json
askTRP-allcomments-2020-08-19_106.json
askTRP-allcomments-2020-08-19_107.json
askTRP-allcomments-2020-08-19_108.json
askTRP-allcomments-2020-08-19_109.json
askTRP-allcomments-2020-08-19_11.json
askTRP-allcomments-2020-08-19_110.json
askTRP-allcomments-2020-08-19_111.json
askTRP-allcomments-2020-08-19_112.json
askTRP-allcomments-2020-08-19_113.json
askTRP-allcomments-2020-08-19_114.json
askTRP-allcomments-2020-08-19_115.json
askTRP-allcomments-2020-08-19_116.json
askTRP-allcomments-2020-08-19_117.json
askTRP-allcomments-2020-08-19_118.json
askTRP-allcomments-2020-08-19_119.json
askTRP-allcomments-2020-08-19_12.json
askTRP-allcomments-2020-08-19_120.json
askTRP-allcomments-2020-08-19_121.json
askTRP-allcomments-2020-08-1

badwomensanatomy-allcomments-2020-08-20_10.json
badwomensanatomy-allcomments-2020-08-20_11.json
badwomensanatomy-allcomments-2020-08-20_12.json
badwomensanatomy-allcomments-2020-08-20_13.json
badwomensanatomy-allcomments-2020-08-20_14.json
badwomensanatomy-allcomments-2020-08-20_15.json
badwomensanatomy-allcomments-2020-08-20_16.json
badwomensanatomy-allcomments-2020-08-20_17.json
badwomensanatomy-allcomments-2020-08-20_18.json
badwomensanatomy-allcomments-2020-08-20_19.json
badwomensanatomy-allcomments-2020-08-20_2.json
badwomensanatomy-allcomments-2020-08-20_20.json
badwomensanatomy-allcomments-2020-08-20_21.json
badwomensanatomy-allcomments-2020-08-20_22.json
badwomensanatomy-allcomments-2020-08-20_23.json
badwomensanatomy-allcomments-2020-08-20_24.json
badwomensanatomy-allcomments-2020-08-20_25.json
badwomensanatomy-allcomments-2020-08-20_26.json
badwomensanatomy-allcomments-2020-08-20_27.json
badwomensanatomy-allcomments-2020-08-20_28.json
badwomensanatomy-allcomments-2020-08-20_2

Creation of the directory data/raw/comments/mgtowbooks-allcomments-2020-08-21 failed
No directory created
raw_comments = data/raw/comments/mgtowbooks-allcomments-2020-08-21.json
mgtowbooks-allcomments-2020-08-21_1.json
mgtowbooks-allcomments-2020-08-21_1.json
data/raw/comments/mgtowbooks-allcomments-2020-08-21.json end
data/raw/comments/pua-allcomments-2020-08-21.json
Creation of the directory data/raw/comments/pua-allcomments-2020-08-21 failed
No directory created
raw_comments = data/raw/comments/pua-allcomments-2020-08-21.json
pua-allcomments-2020-08-21_1.json
pua-allcomments-2020-08-21_1.json
data/raw/comments/pua-allcomments-2020-08-21.json end
data/raw/comments/redpillbooks-allcomments-2020-08-20.json
Creation of the directory data/raw/comments/redpillbooks-allcomments-2020-08-20 failed
No directory created
raw_comments = data/raw/comments/redpillbooks-allcomments-2020-08-20.json
redpillbooks-allcomments-2020-08-20_1.json
redpillbooks-allcomments-2020-08-20_1.json
data/raw/comment

seduction-allcomments-2020-08-21_44.json
seduction-allcomments-2020-08-21_45.json
seduction-allcomments-2020-08-21_46.json
seduction-allcomments-2020-08-21_47.json
seduction-allcomments-2020-08-21_48.json
seduction-allcomments-2020-08-21_49.json
seduction-allcomments-2020-08-21_5.json
seduction-allcomments-2020-08-21_50.json
seduction-allcomments-2020-08-21_51.json
seduction-allcomments-2020-08-21_52.json
seduction-allcomments-2020-08-21_53.json
seduction-allcomments-2020-08-21_54.json
seduction-allcomments-2020-08-21_55.json
seduction-allcomments-2020-08-21_56.json
seduction-allcomments-2020-08-21_57.json
seduction-allcomments-2020-08-21_58.json
seduction-allcomments-2020-08-21_59.json
seduction-allcomments-2020-08-21_6.json
seduction-allcomments-2020-08-21_60.json
seduction-allcomments-2020-08-21_61.json
seduction-allcomments-2020-08-21_62.json
seduction-allcomments-2020-08-21_63.json
seduction-allcomments-2020-08-21_64.json
seduction-allcomments-2020-08-21_65.json
seduction-allcomme

Creation of the directory data/raw/comments/TumblrInAction-allcomments-2020-09-16 failed
No directory created
raw_comments = /Volumes/SAMSUNG/trpred/data/raw/comments/TumblrInAction-allcomments-2020-09-16.json
TumblrInAction-allcomments-2020-09-16_1.json
TumblrInAction-allcomments-2020-09-16_10.json
TumblrInAction-allcomments-2020-09-16_100.json
TumblrInAction-allcomments-2020-09-16_101.json
TumblrInAction-allcomments-2020-09-16_102.json
TumblrInAction-allcomments-2020-09-16_103.json
TumblrInAction-allcomments-2020-09-16_104.json
TumblrInAction-allcomments-2020-09-16_105.json
TumblrInAction-allcomments-2020-09-16_106.json
TumblrInAction-allcomments-2020-09-16_107.json
TumblrInAction-allcomments-2020-09-16_108.json
TumblrInAction-allcomments-2020-09-16_109.json
TumblrInAction-allcomments-2020-09-16_11.json
TumblrInAction-allcomments-2020-09-16_110.json
TumblrInAction-allcomments-2020-09-16_111.json
TumblrInAction-allcomments-2020-09-16_112.json
TumblrInAction-allcomments-2020-09-16_113.

TumblrInAction-allcomments-2020-09-16_109.json
TumblrInAction-allcomments-2020-09-16_11.json
TumblrInAction-allcomments-2020-09-16_110.json
TumblrInAction-allcomments-2020-09-16_111.json
TumblrInAction-allcomments-2020-09-16_112.json
TumblrInAction-allcomments-2020-09-16_113.json
TumblrInAction-allcomments-2020-09-16_114.json
TumblrInAction-allcomments-2020-09-16_115.json
TumblrInAction-allcomments-2020-09-16_116.json
TumblrInAction-allcomments-2020-09-16_117.json
TumblrInAction-allcomments-2020-09-16_118.json
TumblrInAction-allcomments-2020-09-16_119.json
TumblrInAction-allcomments-2020-09-16_12.json
TumblrInAction-allcomments-2020-09-16_120.json
TumblrInAction-allcomments-2020-09-16_121.json
TumblrInAction-allcomments-2020-09-16_122.json
TumblrInAction-allcomments-2020-09-16_123.json
TumblrInAction-allcomments-2020-09-16_124.json
TumblrInAction-allcomments-2020-09-16_125.json
TumblrInAction-allcomments-2020-09-16_126.json
TumblrInAction-allcomments-2020-09-16_127.json
TumblrInAction-

In [66]:
comment_files[4:]

['data/raw/comments/KotakuInAction_allcomments_10-06.json',
 'data/raw/comments/masculism-allcomments-2020-10-03.json',
 'data/raw/comments/MensRights-allcomments-2020-10-02.json',
 'data/raw/comments/MGTOW-allcomments-2020-10-03.json',
 'data/raw/comments/PurplePillDebate-allcomments-2020-10-03.json',
 'data/raw/comments/TheRedPill-allcomments-2020-10-01.json']

In [63]:
test = "data/raw/comments/KotakuInAction_allcomments_10-06.json"
regex = r"([^\/]+)(?=.all)"
matches = re.search(regex, test)
new_file = matches.group(1)

In [64]:
new_file

'KotakuInAction'

In [21]:
test_new = dt.datetime.fromtimestamp(test).date()
type(test_new)

datetime.date

In [82]:
def clean_submissions(raw_submissions):
    """Cleans the file passed in.
    Saves files to processed/submissions folder within trpred."""

    # Get name for processed file
    regex = r"([^\/]+)(?=.allsubmissions)"
    matches = re.search(regex, raw_submissions)
    new_file = matches.group(1)

    # Create list of columns to keep
    keep_cols = ['id', 'created_utc', 'author', 'title',\
                  'score', 'num_comments', 'subreddit', 'link_flair_text']

    keep_cols_text = ['id', 'created_utc', 'author', 'selftext']

    # Create file name
    processedfile_csv = "data/processed/submissions/" + new_file + \
        "-metadata" +  ".csv"

    processed_textfile_csv = "data/processed/submissions/" + new_file + \
        "-text" + ".csv"

    # Create empty data frame
    df_keep = pd.DataFrame()
    df_keep_text = pd.DataFrame()

    # Read in json file
    try:
        data = pd.read_json(raw_submissions)

    # ValueError: Trailing data thrown if file is pretty indented
    except ValueError:
        data = pd.read_json(raw_submissions, lines = True)

    try:
        df_keep = df_keep.append(data[keep_cols])
    except KeyError:
        keep_cols = ['id', 'created_utc', 'author', 'title',\
                    'score', 'num_comments', 'subreddit']
        df_keep = df_keep.append(data[keep_cols])

    try:
        df_keep_text = df_keep_text.append(data[keep_cols_text])
    except KeyError:
        keep_cols_text = ['id', 'created_utc', 'author']
        df_keep_text = df_keep_text.append(data[keep_cols_text])


    # Change date format
    ## For metadata
    df_keep['datetime_dv'] = pd.to_datetime(df_keep['created_utc'], unit = 's')# dv = derived
    df_keep['date_dv'] = df_keep['datetime_dv'].dt.date

    # For text
    df_keep_text['datetime_dv'] = pd.to_datetime(df_keep_text['created_utc'], unit = 's')# dv = derived
    df_keep_text['date_dv'] = df_keep_text['datetime_dv'].dt.date


    ##### Delimit by date #####
    start = dt.datetime.fromtimestamp(utils.get_startdate()).date()
    end = dt.datetime.fromtimestamp(utils.get_enddate()).date()

    # Create mask of time slot
    mask = (df_keep['date_dv'] >= start) & (df_keep['date_dv'] <= end) # inclusive on either end
    mask_text = (df_keep_text['date_dv'] >= start) & (df_keep_text['date_dv'] <= end)

    # Only keep data within date frame
    df_keep = df_keep.loc[mask]
    df_keep_text = df_keep_text.loc[mask_text]
    ############################


    # Save to json
    df_keep_text.to_csv(processed_textfile_csv, mode = "w")
    df_keep.to_csv(processedfile_csv, mode = "w") # mode= w will overwrite previous file


    data = [] # force empty

In [3]:
import glob 

submission_files = ["data/raw/submissions/100sets-allsubmissions-2020-09-22.json",
"data/raw/submissions/askanincel-allsubmissions-2020-09-22.json",
"data/raw/submissions/AskFeminists-allsubmissions-2020-09-23.json",
"data/raw/submissions/BlackPillScience-allsubmissions-2020-09-22.json",
"data/raw/submissions/Braincels-allsubmissions-2020-09-22.json",
"data/raw/submissions/fappeningdiscussion-allsubmissions-2020-09-23.json",
"data/raw/submissions/fPUA-allsubmissions-2020-09-23.json",
"data/raw/submissions/IncelTears-allsubmissions-2020-09-22.json",
"data/raw/submissions/KotakuInAction-allsubmissions-2020-09-27.json",
"data/raw/submissions/LeftWingMaleAdvocates-allsubmissions-2020-09-12.json",
"data/raw/submissions/MGTOW2-allsubmissions-2020-09-27.json",
"data/raw/submissions/MGTOWmusic-allsubmissions-2020-09-12.json",
"data/raw/submissions/MRAmemes-allsubmissions-2020-09-12.json",
"data/raw/submissions/RedPillReadingGroup-allsubmissions-2020-09-27.json",
"data/raw/submissions/RedPillScience-allsubmissions-2020-09-22.json",
"data/raw/submissions/TumblrInAction-allsubmissions-2020-09-27.json",
"data/raw/submissions/WhereAreAllTheGoodMen-allsubmissions-2020-09-27.json"]

all_subs = glob.glob("data/raw/submissions/*.json")

In [4]:
remaining = [x for x in all_subs if x not in submission_files]

remaining

['data/raw/submissions/RedPillLit-allsubmissions-2020-08-13.json',
 'data/raw/submissions/TRPOffTopic-allsubmissions-2020-08-13.json',
 'data/raw/submissions/MRActivism-allsubmissions-2020-08-19.json',
 'data/raw/submissions/redpillfatherhood-allsubmissions-2020-08-13.json',
 'data/raw/submissions/thankTRP-allsubmissions-2020-08-13.json',
 'data/raw/submissions/NOMAAM-allsubmissions-2020-08-19.json',
 'data/raw/submissions/TRPmemes-allsubmissions-2020-08-13.json',
 'data/raw/submissions/marriedredpill-allsubmissions-2020-08-13.json',
 'data/raw/submissions/MensRightsMeta-allsubmissions-2020-08-15.json',
 'data/raw/submissions/MGTOW-allsubmissions-2020-08-23.json',
 'data/raw/submissions/AskFeminists-allsubmissions-2020-09-27.json',
 'data/raw/submissions/RedPillWives-allsubmissions-2020-08-13.json',
 'data/raw/submissions/theredpillright-allsubmissions-2020-08-13.json',
 'data/raw/submissions/badwomensanatomy-allsubmissions-2020-08-13.json',
 'data/raw/submissions/PurplePillDebate-alls

In [ ]:
os.chdir("/Volumes/SAMSUNG/trpred")

for i in remaining:
    clean_submissions(i)
    print(i + " done")


1383
data/processed/submissions/Egalitarianism-text.csv
data/raw/submissions/Egalitarianism-allsubmissions-2020-08-13.json done
2
data/processed/submissions/EthnicRedPill-text.csv
data/raw/submissions/EthnicRedPill-allsubmissions-2020-08-13.json done
11826
data/processed/submissions/FeMRADebates-text.csv
data/raw/submissions/FeMRADebates-allsubmissions-2020-08-14.json done
138
data/processed/submissions/GEOTRP-text.csv
data/raw/submissions/GEOTRP-allsubmissions-2020-08-14.json done
199
data/processed/submissions/IncelsInAction-text.csv
data/raw/submissions/IncelsInAction-allsubmissions-2020-08-14.json done
1819
data/processed/submissions/IncelsWithoutHate-text.csv
data/raw/submissions/IncelsWithoutHate-allsubmissions-2020-08-14.json done
133025
data/processed/submissions/MGTOW-text.csv
data/raw/submissions/MGTOW-allsubmissions-2020-08-23.json done
407
data/processed/submissions/MRActivism-text.csv
data/raw/submissions/MRActivism-allsubmissions-2020-08-19.json done
2260
data/processed/s